In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pathlib

In [2]:
path_to_file = 'michael_scripts.txt'

### Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 844866 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

All right Jim. Your quarterlies look very good. How are things at the library? So you've come to the master for guidance? Is this what you're saying, grasshopper? All right. Well, let me show you how it's done. So that's the way it's done. I've, uh, 


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

84 unique characters


Now create the `tf.keras.layers.StringLookup` layer:

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(844866,), dtype=int64, numpy=array([31, 70, 70, ..., 59, 71, 14])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [13]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

A
l
l
 
r
i
g
h
t
 


In [14]:
seq_length = 150
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'A' b'l' b'l' b' ' b'r' b'i' b'g' b'h' b't' b' ' b'J' b'i' b'm' b'.'
 b' ' b'Y' b'o' b'u' b'r' b' ' b'q' b'u' b'a' b'r' b't' b'e' b'r' b'l'
 b'i' b'e' b's' b' ' b'l' b'o' b'o' b'k' b' ' b'v' b'e' b'r' b'y' b' '
 b'g' b'o' b'o' b'd' b'.' b' ' b'H' b'o' b'w' b' ' b'a' b'r' b'e' b' '
 b't' b'h' b'i' b'n' b'g' b's' b' ' b'a' b't' b' ' b't' b'h' b'e' b' '
 b'l' b'i' b'b' b'r' b'a' b'r' b'y' b'?' b' ' b'S' b'o' b' ' b'y' b'o'
 b'u' b"'" b'v' b'e' b' ' b'c' b'o' b'm' b'e' b' ' b't' b'o' b' ' b't'
 b'h' b'e' b' ' b'm' b'a' b's' b't' b'e' b'r' b' ' b'f' b'o' b'r' b' '
 b'g' b'u' b'i' b'd' b'a' b'n' b'c' b'e' b'?' b' ' b'I' b's' b' ' b't'
 b'h' b'i' b's' b' ' b'w' b'h' b'a' b't' b' ' b'y' b'o' b'u' b"'" b'r'
 b'e' b' ' b's' b'a' b'y' b'i' b'n' b'g' b',' b' ' b'g'], shape=(151,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [16]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"All right Jim. Your quarterlies look very good. How are things at the library? So you've come to the master for guidance? Is this what you're saying, g"
b"rasshopper? All right. Well, let me show you how it's done. So that's the way it's done. I've, uh, I've been at Dunder Mifflin for 12 years, the last f"
b'our as Regional Manager. If you want to come through here... See we have the entire floor. So this is my kingdom, as far as the eye can see. This is ou'
b"r receptionist, Pam. Pam! Pam-Pam! Pam Beesly. Pam has been with us for... forever. Right, Pam? If you think she's cute now, you should have seen her a"
b" couple of years ago. Any messages? Oh! Pam, this is from Corporate. How many times have I told you? There's a special filing cabinet for things from c"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Below is a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(3):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"All right Jim. Your quarterlies look very good. How are things at the library? So you've come to the master for guidance? Is this what you're saying, "
Target: b"ll right Jim. Your quarterlies look very good. How are things at the library? So you've come to the master for guidance? Is this what you're saying, g"
Input : b"rasshopper? All right. Well, let me show you how it's done. So that's the way it's done. I've, uh, I've been at Dunder Mifflin for 12 years, the last "
Target: b"asshopper? All right. Well, let me show you how it's done. So that's the way it's done. I've, uh, I've been at Dunder Mifflin for 12 years, the last f"
Input : b'our as Regional Manager. If you want to come through here... See we have the entire floor. So this is my kingdom, as far as the eye can see. This is o'
Target: b'ur as Regional Manager. If you want to come through here... See we have the entire floor. So this is my kingdom, as far as the eye can see. This is ou'


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 150), dtype=tf.int64, name=None), TensorSpec(shape=(64, 150), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 85) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `150` but the model can be run on inputs of any length:

In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  21760     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  87125     
                                                                 
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [30]:
sampled_indices

array([ 2, 51, 66, 54,  6, 34, 70, 71, 19, 78, 22, 31, 44, 66, 36,  7, 52,
       14, 31, 59, 58, 27, 70, 63, 50, 71,  9, 22, 11, 48, 52, 62, 80, 55,
       22,  0, 42, 73, 11, 11, 20, 72, 12, 23, 27, 23, 48, 49, 75, 14, 66,
       68,  6, 45, 15, 67, 41,  7, 54, 77, 78, 35, 18, 55, 74, 53, 13, 38,
       59,  8, 71, 16, 68,  7, 29, 20, 67, 21, 65,  3,  3, 18, 61, 84, 18,
       64, 48, 84, 71, 12, 45, 75, 50, 21, 42, 70, 22, 32, 29,  6,  4, 19,
       72, 39, 67, 41, 24, 14,  7, 39, 65, 30, 15, 60,  1, 73, 17, 80, 76,
       72, 73, 48,  0, 46, 33, 17, 77,  2,  2,  0, 30, 43, 66, 30, 80,  4,
       49, 67, 58, 40, 71,  9, 16, 62,  2, 12, 70, 40, 77, 18])

Decode these to see the text predicted by this untrained model:

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ose pills. You are cured. Actually, you should... better hold onto them pills, just in case. Yes Michael Scott for Dr. Jocelyn, please. Hello doctor I'

Next Char Predictions:
 b'!UhX&Dlm3t6ANhF\'V.Aa];leTm)6+RVdvY6[UNK]Lo++4n,7;7RSq.hj&O/iK\'XstE2YpW-Ha(m0j\'?4i5g""2cz2fRzm,OqT5Ll6B?&$3nIiK8.\'Ig@/b o1vrnoR[UNK]PC1s!![UNK]@Mh@v$Si]Jm)0d!,lJs2'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 150, 85)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4431167, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [34]:
tf.exp(example_batch_mean_loss).numpy()

85.039566

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [35]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', restore_best_weights=True, patience=40)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 500

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback,early_stop])

Epoch 1/500
87/87 [==============================] - 7s 59ms/step - loss: 3.0735 - accuracy: 0.2387
Epoch 2/500
87/87 [==============================] - 6s 58ms/step - loss: 2.2447 - accuracy: 0.3642
Epoch 3/500
87/87 [==============================] - 6s 59ms/step - loss: 1.9837 - accuracy: 0.4297
Epoch 4/500
87/87 [==============================] - 6s 59ms/step - loss: 1.7779 - accuracy: 0.4842
Epoch 5/500
87/87 [==============================] - 6s 59ms/step - loss: 1.6256 - accuracy: 0.5240
Epoch 6/500
87/87 [==============================] - 6s 59ms/step - loss: 1.5140 - accuracy: 0.5527
Epoch 7/500
87/87 [==============================] - 6s 60ms/step - loss: 1.4301 - accuracy: 0.5740
Epoch 8/500
87/87 [==============================] - 6s 60ms/step - loss: 1.3660 - accuracy: 0.5903
Epoch 9/500
87/87 [==============================] - 6s 58ms/step - loss: 1.3141 - accuracy: 0.6036
Epoch 10/500
87/87 [==============================] - 6s 59ms/step - loss: 1.2696 - accuracy: 0.6156

## Generate text

The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["Pam "])
result = [next_char]

for n in range(150):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Pam doing chatting with you? No! Guh! OW! Awww, help, help me! I fell off the toilet. I'm caught between the toilet and the wall. Ugh, never happened. Uh, 

________________________________________________________________________________

Run time: 0.272127628326416


## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'RNN_Model')
# one_step_reloaded = tf.saved_model.load('RNN_Model')

INFO:tensorflow:Assets written to: RNN_Model/assets


INFO:tensorflow:Assets written to: RNN_Model/assets


In [ ]:
from google.colab import files
import os

dir_to_zip = 'RNN_Model/' #@param {type: "string"}
output_filename = 'RNN_Model.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>